# Modelo de precios en función de Peso Declarado y Distancia Real : Una exploración

Importante agregar acá las Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import googlemaps
from aux_func import * #funciones auxiliares

pd.options.display.max_rows = 500  #Máximo de filas a mostrar

Hay que agregar una llave. Para simplificar, lo haremos desde un archivo.

In [ ]:
def get_file_contents(filename):
    """ 
        Dado un nombre de archivo, retorna el contenido del archivo
    """
    try:
        with open(filename, 'r') as f:
            # Se asume que contiene solo una línea,
            # Con la API KEY
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' No se encontro" % filename)

Hay que agregar la API KEY

In [ ]:
api_key = get_file_contents('apikey')

Leemos el histórico de las cargas

In [ ]:
df = pd.read_excel('historico_cargas_enviadas.xlsx')

In [ ]:
df.info()

In [ ]:
df.describe()

Tengo una funcion ad hoc para eliminar los valores extremos. El ```docstring```está acá:

In [ ]:
print(is_outlier.__doc__)

In [ ]:
df_marcas = is_outlier(df,features=['peso','precio_cerrado'])

In [ ]:
df_marcas.sample(5,random_state=1)

In [ ]:
df_filtrado = df_marcas[(df_marcas.is_outlier_peso == False)&\
    ((df_marcas.is_outlier_precio_cerrado) == False)]

In [ ]:
df_filtrado

Hay mucho filtrado que hacer:
- Sin precios 
- Sin pesos
- Sin Origen correcto
- Sin Destino correcto

In [ ]:
sin_precio = (df_filtrado.precio_cerrado==0) | (df_filtrado.precio_cerrado.isnull())
sin_peso = (df_filtrado.peso==0) | (df_filtrado.peso.isnull())
sin_origen = (df_filtrado.coordenadas_origen.isna()) | (df_filtrado.coordenadas_origen.isnull())| (df_filtrado.coordenadas_origen.str.len()<=7)
sin_destino = (df_filtrado.coordenadas_destino.isna()) | (df_filtrado.coordenadas_destino.isnull())|(df_filtrado.coordenadas_destino.str.len()<=7)

In [ ]:
df_modelo = df_filtrado[((~sin_precio)&(~sin_peso)&(~sin_origen)&(~sin_destino))]

In [ ]:
df_modelo

Instancio un objeto Cliente de GMaps

In [ ]:
gmaps = googlemaps.Client(key=api_key)

Algunas pruebas:

In [ ]:
a = (-33.442293,-70.625701)
b = (-33.379269,-70.691431)

In [ ]:
my_dist = gmaps.distance_matrix(a,b)['rows'][0]['elements'][0]

In [ ]:
my_dist['distance']['value']

Tengo que crear una función de distancia que usa información de googlemaps

In [ ]:
def distancia (origin, destination):
    """
    Dado un origen y un destino, retorna la distancia en metros.
    """
    try: 
        my_dist = gmaps.distance_matrix(origin,destination)
        return int(round(1.0/10000* my_dist['rows'][0]['elements'][0]['distance']['value'],0))
    except:
        return 1e12 #Censuro con un valor absurdo datos mal imputados


Y luego vectorizarla y expresar en km

In [ ]:
v_distancia = np.vectorize(distancia)

In [ ]:
distancias = v_distancia(df_modelo.coordenadas_origen,df_modelo.coordenadas_destino)

In [ ]:
df_modelo = df_modelo.assign(distancia_km = distancias)

In [ ]:
df_modelo = df_modelo[(df_modelo.distancia_km>0)&(df_modelo.distancia_km<=1e6)]
df_modelo = df_modelo[['peso','precio_cerrado','distancia_km']]
df_modelo = df_modelo[df_modelo.precio_cerrado>=10000]

In [ ]:
df_modelo

In [ ]:
from pycaret.regression import *

In [ ]:
exp_reg101 = setup(data = df_modelo, 
                   target = 'precio_cerrado', 
                   session_id=123)

Importo algunas bibliotecas para ajustar modelos, nada muy sofisticado por el momento

In [ ]:
compare_models()

In [ ]:
from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor

In [ ]:
X = df_modelo.drop(['precio_cerrado'],axis=1)
y = df_modelo.precio_cerrado

In [ ]:
y

In [ ]:
lr = LinearRegression()
cb = CatBoostRegressor()

In [ ]:
lr.fit(X,y)

In [ ]:
lr.coef_

In [ ]:
lr.intercept_

In [ ]:
X.head()

In [ ]:
cb.fit(X,y,verbose=False)

In [ ]:
cb.feature_importances_

In [ ]:
y_pred_lr = lr.predict(X)
y_pred_cb = cb.predict(X)

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [ ]:
mae_lineal = mean_absolute_error(y,y_pred_lr)
mse_lineal = mean_squared_error(y,y_pred_lr)
mae_cb = mean_absolute_error(y,y_pred_cb)
mse_cb = mean_squared_error(y,y_pred_cb)


print(f"El MAE del modelo lineal es {round(mae_lineal,0)}")
print(f"El MAE del modelo catboost es {round(mae_cb,0)}")


print(f"El MSE del modelo lineal es {round(mse_lineal,0)}")
print(f"El MSE del modelo catboost es {round(mse_cb,0)}")

Notar que predicciones con ```CatBoostRegressor``` son más precisas que las del modelo lineal.

In [ ]:
predicciones = pd.DataFrame(zip(y,y_pred_cb,round(y-y_pred_cb,0),df_modelo.peso,df_modelo.distancia_km),
columns=['precio_cerrado','prediccion_cb','error_cb','peso_kg','distancia_km'])
predicciones.prediccion_cb = predicciones.prediccion_cb.round(0) 

In [ ]:
predicciones

## Siguientes pasos

- Mejorar histórico de cargas : mucho registro con errores
- Tener más registros para separar en ```entrenamiento``` y ```prueba```
- Afinar coeficientes   


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(y.values,y_pred_cb,alpha=0.6)
plt.plot(y.values,y.values,'r','--')
plt.xlim(0,70e3)
plt.ylim(0,70e3)
plt.show()

In [ ]:
y.values